# Buat Dataset

In [3]:
import pandas as pd

In [73]:
df = pd.read_excel('pos_model.xlsx')

In [111]:
df_encode = df.groupby('SentenceID')['Word'].apply(list).reset_index()
df_encode.columns = ['SentenceID', 'Sentence']

In [113]:
df_encode.head()

,SentenceID,Sentence
0,0,"[Pan, Karsa, ajaka, pianakne, muani, nanggap, ..."
1,1,"[Uli, semeng, maka, sanja, ia, ajaka, dadua, t..."
2,2,"[Kenehne, apang, gegaene, enggal, pragat, tur,..."
3,3,"[Telung, dina, ia, magae, tan, parerenan, seme..."
4,4,"[Kendelne, tara, bakat, ban, nuturang]"


ubah urutan sentence id

In [76]:
index_sentence = 0
sentence_id = []

for index, data in df.iterrows():
    if index == len(df)-1:
        print('finish')
    elif df['SentenceID'][index] != df['SentenceID'][index+1]:
        index_sentence += 1
    sentence_id.append(index_sentence)

finish


hapus titik dan unidecode

In [77]:
df['SentenceID'] = sentence_id

In [78]:
df = df[df['Word'] != '.']
df = df[df['Word'] != ',,']
from unidecode import unidecode

# Misalkan DataFrame Anda bernama df
df['Word'] = df['Word'].apply(unidecode)

In [87]:
df.tail()

,StoryID,SentenceID,Word,new POS
89911,164,6635,pecut,propn
89912,164,6635,mase,propn
89913,164,6635,pejanga,verb
89914,164,6635,di,adp
89915,164,6635,tendasne,noun


In [88]:
df.to_excel('pos_model_2.xlsx', index=False)

In [80]:
df = pd.read_excel('pos_model_2.xlsx')

# Word2Vec


In [81]:
import random

def training_data(sentence, vocabulary, WINDOW_SIZE=5, NEGATIVE_SAMPLES=5):
    data = []

    # Iterasi untuk setiap kata dalam kalimat
    for index_center_word, center_word in enumerate(sentence):
        # Mengambil kata konteks dalam jendela
        context_words = []
        for window in range(1, WINDOW_SIZE + 1):
            if index_center_word + window < len(sentence):
                context_words.append(sentence[index_center_word + window])
            if index_center_word - window >= 0:
                context_words.append(sentence[index_center_word - window])

        # Untuk setiap kata konteks, buat pasangan positif
        for context_word in context_words:
            data.append([center_word, context_word, 1])

        # Untuk negative sampling, pilih kata acak dari vocabulary yang tidak ada dalam jendela
        # Misalnya, jika vocabulary berisi daftar semua kata dalam corpus
        for _ in range(NEGATIVE_SAMPLES):
            negative_sample = random.choice(vocabulary)
            while negative_sample in context_words or negative_sample == center_word:
                negative_sample = random.choice(vocabulary)
            data.append([center_word, negative_sample, 0])

    return data


### vocabulary

In [114]:
vocab = list(set(df['Word'].tolist()))

In [115]:
len(vocab)

12899

In [116]:
data_skip_gram = []

for index, data in df_encode.iterrows():
    data_skip_gram.extend(training_data(sentence=data['Sentence'], vocabulary=vocab))

In [117]:
len(data_skip_gram)

1055896

In [118]:
import numpy as np

df_skip_gram = pd.DataFrame(columns=['center_word', 'context_word', 'label'], data=data_skip_gram)
words = np.intersect1d(df_skip_gram.context_word, df_skip_gram.center_word)
df_skip_gram = df_skip_gram[(df_skip_gram.center_word.isin(words)) & (df_skip_gram.context_word.isin(words))].reset_index(drop=True)

In [119]:
words

array(['310', '90', 'Aa', ..., 'yuta', 'yuyu', 'yuyune'], dtype=object)

### Fungsi

In [120]:
def sigmoid(v, scale=1):
    return 1 / (1 + np.exp(-scale*v))

In [121]:
def normalize_data(data):
    row_norms = np.sqrt((data.values**2).sum(axis=1)).reshape(-1,1)
    return data.divide(row_norms, axis='index')

In [122]:
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

def plot_words(debug):
    plt.figure(figsize=(8,4))
    
    plt.subplot(1,2,1)
    lim_main_first = main_embeddings.loc[[debug[0]]]
    lim_main_second = main_embeddings.loc[[debug[1]]]
    p1 = plt.scatter(lim_main_first[0], lim_main_first[1], color='r')
    plt.arrow(0,0,float(lim_main_first[0]), float(lim_main_first[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_main_first.iterrows():
        plt.text(row[0], row[1], str(idx))
    p2 = plt.scatter(lim_main_second[0], lim_main_second[1], color='r')
    plt.arrow(0,0,float(lim_main_second[0]), float(lim_main_second[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_main_second.iterrows():
        plt.text(row[0], row[1], str(idx))
    sim = 1 - cosine(main_embeddings.loc[debug[0]], main_embeddings.loc[debug[1]])
    plt.title('Sim = %s'%round(sim,4), fontsize=20)
    plt.axvline(0, color='k', linestyle='--', alpha=0.5)
    plt.axhline(0, color='k', linestyle='--', alpha=0.5)
    
    t = np.arange(0, 3.14*2+0.1, 0.1)
    plt.plot(np.cos(t), np.sin(t), linewidth=1, color='k', alpha=0.5, linestyle='--')
    
    ###################################
    
    plt.subplot(1,2,2)
    lim_main = main_embeddings.loc[[debug[0]]]
    lim_context = context_embeddings.loc[[debug[1]]]
    p1 = plt.scatter(lim_main[0], lim_main[1], color='r')
    plt.arrow(0,0,float(lim_main[0]), float(lim_main[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_main.iterrows():
        plt.text(row[0], row[1], str(idx))
    p2 = plt.scatter(lim_context[0], lim_context[1], color='b')
    plt.arrow(0,0,float(lim_context[0]), float(lim_context[1]), head_width=0.01, length_includes_head=True)
    for idx,row in lim_context.iterrows():
        plt.text(row[0], row[1], str(idx))
    sim = 1 - cosine(main_embeddings.loc[debug[0]], context_embeddings.loc[debug[1]])
    plt.title('Sim = %s'%round(sim,4), fontsize=20)
    plt.axvline(0, color='k', linestyle='--', alpha=0.5)
    plt.axhline(0, color='k', linestyle='--', alpha=0.5)
    
    plt.plot(np.cos(t), np.sin(t), linewidth=1, color='k', alpha=0.5, linestyle='--')
    
    plt.show()

In [123]:
def update_embeddings(df, main_embeddings, context_embeddings, learning_rate, debug=False):
    
    #mencari perbedaan(diffs) antara main_embedding dengan context_embedding
    main_embeddings_center = main_embeddings.loc[df.center_word].values
    context_embeddings_context = context_embeddings.loc[df.context_word].values
    diffs = context_embeddings_context - main_embeddings_center
    
    #Mencari similaritas, skor, dan error diantaran main_embedding dengan context_embedding
    dot_products = np.sum(main_embeddings_center * context_embeddings_context, axis=1)
    scores = sigmoid(dot_products)
    errors = (df.label - scores).values.reshape(-1,1)
    
    #calculate updates
    updates = diffs*errors*learning_rate
    updates_df = pd.DataFrame(data=updates)
    updates_df['center_word'] = df.center_word
    updates_df['context_word'] = df.context_word
    updates_df_center = updates_df.groupby('center_word').sum(numeric_only=True)
    updates_df_context = updates_df.groupby('context_word').sum(numeric_only=True)
    
    if debug:
        plot_words(debug)   

    #apply updates
    main_embeddings += updates_df_center.loc[main_embeddings.index]
    context_embeddings -= updates_df_context.loc[context_embeddings.index]
    
    #normalize embeddings
    main_embeddings = normalize_data(main_embeddings)
    context_embeddings = normalize_data(context_embeddings)
    
    #return the updated embeddings
    return main_embeddings, context_embeddings

In [124]:
EMBEDDING_SIZE = 100

main_embeddings = np.random.normal(0,0.1,(len(words), EMBEDDING_SIZE))
row_norms = np.sqrt((main_embeddings**2).sum(axis=1)).reshape(-1,1)
main_embeddings = main_embeddings / row_norms

context_embeddings = np.random.normal(0,0.1,(len(words), EMBEDDING_SIZE))
row_norms = np.sqrt((context_embeddings**2).sum(axis=1)).reshape(-1,1)
context_embeddings = context_embeddings / row_norms

main_embeddings = pd.DataFrame(data=main_embeddings, index=words)
context_embeddings = pd.DataFrame(data=context_embeddings, index=words)

epoch = 25

for _ in range(epoch):
    main_embeddings, context_embeddings = update_embeddings(df_skip_gram, main_embeddings, context_embeddings, 0.01)

In [125]:
# Menggabungkan nilai seluruh kolom pada setiap baris menjadi list
main_embeddings['vektor_embedding'] = main_embeddings.apply(lambda row: row.tolist(), axis=1)

# Membuat DataFrame baru dengan kolom 'combined' yang berisi list dari seluruh kolom
final_word_embedding = main_embeddings[['vektor_embedding']]


In [126]:
final_word_embedding.head()

,vektor_embedding
310,"[0.008706084013628216, -0.04113459350431811, 0..."
90,"[-0.16363709820048164, -0.143370451198932, 0.0..."
Aa,"[-0.11810708768793127, 0.03206814061440846, -0..."
Abimanyu,"[-0.05830379906390936, -0.04773073165083823, -..."
Abulan,"[0.21767462207887103, 0.1179176898254253, -0.0..."


In [127]:
final_word_embedding.to_excel('final_word_embedding.xlsx', index=True)

## aplikasi word embedding ke dataset

In [128]:
final_word_embedding = pd.read_excel('final_word_embedding.xlsx')

In [ ]:
final_word_embedding = final_word_embedding.rename(columns={'Unnamed: 0' : 'word'})

In [183]:
def data_embedding_funct(word):
    return final_word_embedding.loc[final_word_embedding['word'] == word, 'vektor_embedding'].values

In [134]:
df['word_emb'] = df['Word'].apply(data_embedding_funct)

In [278]:
df.head()

,StoryID,SentenceID,Word,new POS,word_emb
0,0,0,Pan,propn,"[[0.07002387715406538, -0.028471471947595483, ..."
1,0,0,Karsa,punct,"[[0.014792937983094003, -0.13909075156172937, ..."
2,0,0,ajaka,adp,"[[0.07034387839105, -0.0277573743620209, 0.135..."
3,0,0,pianakne,verb,"[[0.05553844200859273, -0.03205081261100487, 0..."
4,0,0,muani,noun,"[[0.05201503156493778, -0.03286708678576444, 0..."


In [121]:
df.to_excel('pos_model_final.xlsx', index=False)

In [ ]:
def join_token(df, pos_tags=[]):
    temp_sentence_id = 0

    sentence = []
    sentences = []

    word_emb = []
    word_embs = []

    pos_tag = []
    pos_tags = []

    for index, data in df.iterrows():
        if temp_sentence_id == df['SentenceID'][index]:
            sentence.append(data['Word'])
            word_emb.append(data['word_emb'])
            pos_tag.append(data['new POS'])
        else:
            sentences.append(sentence)
            word_embs.append(word_emb)
            pos_tags.append(pos_tag)

            sentence = []
            word_emb = []
            pos_tag = []

            temp_sentence_id += 1
            sentence.append(data['Word'])
            word_emb.append(data['word_emb'])
            pos_tag.append(data['new POS'])
    return sentences, word_embs, pos_tags
            

In [25]:
sentences, word_embs, pos_tags = join_token(df)
data = {
    'sentences' : sentences,
    'word_embs' : word_embs,
    'pos_tags' : pos_tags
}

df_dataset = pd.DataFrame(data)

In [256]:
print(type(df_dataset['word_embs'][0][0]))
print(len(df_dataset['word_embs'][0][0]))
df_dataset['word_embs'][0][0]

<class 'str'>
2174


'[0.12964028080210996, -0.09584557628465226, 0.0655824654971349, -0.05066044429784154, 0.10998222827387567, 0.03577038938767248, -0.1739068161416247, -0.08355342057577972, -0.06266904160187911, -0.026727112716117305, 0.005669683763925348, -0.11858280729549119, -0.22678012484483273, 0.038512443663754455, 0.10755820291299106, -0.07513139204035889, 0.09048007251225458, 0.11488925911552383, -0.02034605971750329, -0.09421117343602718, -0.07717109482453818, -0.035327546697538584, -0.15152994399945027, -0.07094829972188972, 0.008988788331132131, -0.027797145897903067, -0.03254293872863036, -0.07480245956082596, -0.053945412856573345, -0.04488172450516051, -0.0040252880507345755, -0.21698266250197726, -0.14118284345194676, 0.12728285834714947, 0.06759178231666081, 0.17882710730796353, -0.017745728343314244, -0.04537996897663905, 0.049403713760017065, -0.130926563371036, -0.008407953293425087, 0.027842612730696062, 0.011415118461265585, -0.021892717795998242, 0.03745760010676721, -0.05823870496

In [27]:
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [272]:
def sentences_to_features(sentence, sentence_embedding):
    features = []

    for index in range(len(sentence)):
        feature = {
            'bias': 1.0,
            'word': sentence[index],
            'word_embedding': sentence_embedding[index],
            'is_first': index == 0,
            'is_last': index == len(sentence) - 1,
            'prev_word': '' if index == 0 else sentence[index-1],
            'next_word': '' if index == len(sentence) - 1 else sentence[index+1]
        }
        features.append(feature)
    return features

In [273]:
def prepare_data(df):
    sentences = []
    labels = []

    for index in range(len(df)):
        # Menyusun fitur untuk CRF dan label POS
        sentences.append(sentences_to_features(df['sentences'][index],df['word_embs'][index]))
        labels.append(df.iloc[index]['pos_tags'])
    
    return sentences, labels

In [30]:
X, y = prepare_data(df_dataset)

# Membagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
X_train[3]

[{'bias': 1.0,
  'word': 'Ring',
  'word_embedding': '[0.13654103264641465, -0.18263010648026187, 0.12618458316822023, -0.0931915282702292, -0.01520771328778107, 0.08128388589394599, -0.0321047338583863, 0.029486606308703797, 0.041175371797387436, -0.0022777913425290836, -0.12773856614865559, 0.07179661461104177, 0.08604576913429092, 0.05889672050934415, 0.03501972865000048, -0.1381303722616548, 0.07538905014199765, -0.20647976413478797, -0.06558730579182812, -0.018050504780568966, -0.012428563056648093, -0.1008260387705123, 0.02690351287989135, 0.1453829211830896, 0.20641369893201528, 0.0520105508373885, -0.11287306744381369, 0.027732946915755872, -0.12843944562926074, -0.012975356894740673, 0.03280997686915463, -0.05599318646000671, -0.04630353458620499, 0.1400842377204049, 0.0021094137518620345, -0.02787887457878065, -0.06177550750014503, -0.11309047709598363, -0.11589041627108224, -0.17129754197272962, 0.2009922008117592, -0.08659923176844932, 0.1401498410578184, 0.0448682815031241

In [ ]:
import pycrfsuite

# Inisialisasi trainer
trainer = pycrfsuite.Trainer(verbose=False)

# Tambahkan data pelatihan
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Atur parameter CRF
trainer.set_params({
    'c1': 1.0,  # Regularisasi L1
    'c2': 1.0,  # Regularisasi L2
    'max_iterations': 100,
    'feature.possible_transitions': True
})

# Melatih model
trainer.train('crf.model')

In [42]:
from sklearn.metrics import classification_report

# Memuat model CRF yang sudah dilatih
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

# Melakukan prediksi pada data uji
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Menghitung akurasi
y_test_flat = [label for seq in y_test for label in seq]  # Flatten y_test
y_pred_flat = [label for seq in y_pred for label in seq]  # Flatten y_pred

# Laporan klasifikasi
print(classification_report(y_test_flat, y_pred_flat))


              precision    recall  f1-score   support

         adj       0.94      0.68      0.79       284
         adp       0.97      0.91      0.94      1650
         adv       0.95      0.85      0.90      1367
         det       0.98      0.96      0.97      1048
        noun       0.87      0.90      0.89      5073
         num       1.00      0.56      0.72        62
        pron       0.96      0.92      0.94       951
       propn       0.78      0.84      0.81      1104
       punct       0.78      0.84      0.81      2168
        verb       0.87      0.87      0.87      2780

    accuracy                           0.88     16487
   macro avg       0.91      0.83      0.86     16487
weighted avg       0.88      0.88      0.88     16487



In [43]:
# Hitung jumlah prediksi yang benar
correct = 0
total = 0

for true_seq, pred_seq in zip(y_test, y_pred):
    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label == pred_label:
            correct += 1
        total += 1

# Akurasi
accuracy = correct / total
print(f"Akurasi: {accuracy:.2%}")

Akurasi: 87.94%


In [44]:
from sklearn.metrics import f1_score

# Menghitung F1-Score
f1 = f1_score(y_test_flat, y_pred_flat, average='weighted')
print(f"F1-Score: {f1:.2%}")

F1-Score: 88.00%


In [276]:
def proses_pos(sentence):

    crf = pycrfsuite.Tagger()
    crf.open('crf.model')

    sentence_embs = []

    sentence_split = sentence.split()

    for word in sentence_split:
        sentence_embs.append(data_embedding_funct(word))

    data_temp = {
        'sentences' : sentence_split,
        'word_embs' : sentence_embs
    }

    df_temp = pd.DataFrame(data_temp)
    df_temp['word_embs'] = df_temp['word_embs'].apply(lambda x: x.tolist())

    feature = sentences_to_features(df_temp['sentences'], df_temp['word_embs'])
    return crf.tag(feature)


In [277]:
sentence = "meme ngomong teken pianakne apanga gegaene aluhan"
print(proses_pos(sentence))

['noun', 'punct', 'adp', 'noun', 'punct', 'verb', 'noun']


In [259]:
print(type(df_dataset['word_embs'][0][0]))
print(len(df_dataset['word_embs'][0][0]))
df_dataset['word_embs'][0][0]

<class 'str'>
2174


'[0.12964028080210996, -0.09584557628465226, 0.0655824654971349, -0.05066044429784154, 0.10998222827387567, 0.03577038938767248, -0.1739068161416247, -0.08355342057577972, -0.06266904160187911, -0.026727112716117305, 0.005669683763925348, -0.11858280729549119, -0.22678012484483273, 0.038512443663754455, 0.10755820291299106, -0.07513139204035889, 0.09048007251225458, 0.11488925911552383, -0.02034605971750329, -0.09421117343602718, -0.07717109482453818, -0.035327546697538584, -0.15152994399945027, -0.07094829972188972, 0.008988788331132131, -0.027797145897903067, -0.03254293872863036, -0.07480245956082596, -0.053945412856573345, -0.04488172450516051, -0.0040252880507345755, -0.21698266250197726, -0.14118284345194676, 0.12728285834714947, 0.06759178231666081, 0.17882710730796353, -0.017745728343314244, -0.04537996897663905, 0.049403713760017065, -0.130926563371036, -0.008407953293425087, 0.027842612730696062, 0.011415118461265585, -0.021892717795998242, 0.03745760010676721, -0.05823870496